# Splitting image with x% overlapping



## 1. Import libraries


In [21]:
from glob import glob
from PIL import Image
import numpy as np
import math

## 2. Define variables


In [28]:
input_dir = "./data"
output_dir = "./output"
# for every two horizontally or vertically adjacent window, the overlapping percentage should be at least {min_overlapping_percentage}%
min_overlapping_percentage = 0.6
# fixed size for sliding window
window_size = (512,512)
eps = 1e5

## 3. Load pictures

In [23]:
img_list = glob(f"{input_dir}/*")
img_cnt = len(img_list)

print(f"Total {img_cnt} pictures")

img_size = Image.open(img_list[0]).size

print(f"Picture size: {img_size}")

Total 132 pictures
Picture size: (5472, 3648)


## 4. Calculate sliding stats

In [24]:
def get_actual_overlapping_percentage(pic_size,window_size):
    # Minimum number of windows to fully cover the picture
    window_per_pic = pic_size/window_size
    min_window_number = math.ceil((window_per_pic-min_overlapping_percentage)/(1-min_overlapping_percentage))
    return (window_per_pic-min_window_number)/(1-min_window_number)
actual_overlapping_percentage = [get_actual_overlapping_percentage(img_size[0],window_size[0]),get_actual_overlapping_percentage(img_size[1],window_size[1])]

print(f"Actual overlapping percentage: {actual_overlapping_percentage[0]} (horizontal), {actual_overlapping_percentage[1]} (vertical)")

Actual overlapping percentage: 0.6125 (horizontal), 0.6171875 (vertical)


## 5. Generate top-left coordinate for windows


In [25]:
def get_coordinate_list(pic_size,actual_overlapping_percentage,window_size):
    coordinate_list = []
    i = 0
    while i < pic_size:
        if round(i*eps)/eps+window_size > pic_size:
            break
        coordinate_list.append(round(i))
        i += window_size*(1-actual_overlapping_percentage)
    return coordinate_list
coordinate_list = [get_coordinate_list(img_size[0],actual_overlapping_percentage[0],window_size[0]),get_coordinate_list(img_size[1],actual_overlapping_percentage[1],window_size[1])]

## 6. Apply on image and write to folder

In [30]:
for img_path in img_list:
    img = Image.open(img_path)
    for x in coordinate_list[0]:
        for y in coordinate_list[1]:
            window = img.crop((x,y,x+window_size[0],y+window_size[1]))
            window.save(f"{output_dir}/{img_path.split('/')[-1].split('.')[0]}_{x}_{y}_{window_size}.png")
    print(f"{img_path} done") 
    break

./data/100_0865_0050.JPG done
